In [ ]:
# data manager and analysis
import vodex as vx
import numan as nu
import pandas as pd

# for saving images
import tifffile as tif
import numpy as np

# DONT TRUST. IT NEEDS TO BE UPDATED. Project structure: 

Provide the project folder with the "processed" folder created in the previous notebook. 

As you keep going with the analysis, the folder will have the following structure: 

```
...........................................................
....................... DONE in 01 ........................
...........................................................
processed                                               ...
│   experiment.json <-----------------------------------... the file that contains everything about the experiment, you are creating it once and will be reusing ever after
│   experiment_dff.json <-------------------------------... everything about the experiment, but loads from the dff movie, not from the raw data                   ...
└───dff_movie  <----------------------------------------...the dff movie :)
│   │   dff_movie_0000.tif                              ...
│   │   dff_movie_0001.tif                              ...
│   │   ...                                             ...
│..........................................................
│...................... DONE in 02 ........................
│..........................................................
│                                                       ...
└───tscore_volumes  <-----------------------------------... t-score tif files
│   │   tscore_SvB.tif <--------------------------------... t-score Stimuli vs Blank
│   │                                                   ...
│..........................................................
│...................... DONE : MANUAL .....................
│..........................................................
└───spots                                               ...
│   └───imaris  <---------------------------- ATTENTION ... You need to put stuff generated by imaris into this folder!!!                                         ...
│       │   └───tscore_SvB_Statistics                   ...
│       │       │     tscore_SvB_Position.csv           ...
│       │       │     tscore_SvB_Diameter.csv           ...
│       │       │     ...                               ...
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++ WILL BE DONE in this notebook +++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│   │                                                   +++
│   └───signals  <--------------------------------------+++ json files with the extracted signals, also will have the group info after you added it                 +++
│       │   spots_SvB.json                              +++
│       │                                               +++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
│   │
│   └───reports  <---------------------------------- tiffs and pdf with the cells significant in any pairwise comparison
│       └───all_significant  <---------------------- tiffs and pdf with all significant in any way cells
│           │   └───signals  <---------------------- pdfs with signals
│           │       │     ...
│           │   └───images <------------------------ tif masks
│           │       │     ...
│       └───groupped  <----------------------------- tiffs and pdf where the cells are groupped based on signal shape .. or anything else you want
│           │   readme.txt  <----------------------- ATTENTION : you need to describe the groups
│           │   └───signals  <---------------------- pdfs with signals
│           │       │     ...
│           │   └───images  <----------------------- tif masks
│           │       │     ...
```

Also, if the processed folder should already exist if you created dff movies. If the folder doesn't exist, it will complain ...

# Make sure you have the imaris files in the right folders... 

You can use any way to segemnt the images, we need position and diameter


```
processed
│!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
│!!!!!!!!!!!!!!!! ATTENTION: MANUAL STEP !!!!!!!!!!!!!!!!!!
│!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
└───spots                                               !!!
│   └───imaris  <---------------------------- ATTENTION !!! You need to put stuff generated by imaris into this folder!!!                                         !!!
│       │   └───tscore_SvB_max_Statistics               !!!
│       │       │     tscore_SvB_max_Position.csv       !!!
│       │       │     tscore_SvB_max_Diameter.csv       !!!
│       │       │     ...                               !!!
│!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
│!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
│!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
```

# Set project folder

In [ ]:
project_folder = "/home/ply/repos/numan/notebooks/vodex_si/data/"
project = nu.Project(project_folder)

project.check_exists("processed")
project.check_exists("processed/spots/imaris")

project.activate("processed")

# Load experiment with the raw data:

In [ ]:
experiment = vx.Experiment.load("experiment_truncated_drift_corrected.db")

## Extract spot signals
This is done for each set of segmentations.

Make the directory for signals:

In [ ]:
project.create("processed/spots/signals")
project.create("processed/spots/masks")

Define constants

In [ ]:
RESOLUTION = [2.4, 1.17, 1.17] # in um, ZYX
Z, Y , X = 60, 296, 540 # in slices, pixels

T_TAGS = ["1vB1","2vB2","3vB3","4vB4","5vB5"]

PAIRING_RADIUS = 3.5 # in um
RADIUS_TAG = "3p5"

# if you run out of memory, reduce the batch size
BATCH_SIZE = 100

Get the wrapper functions

In [ ]:
def load_and_save_signals(spots, group_tag):
    # extract signal ( this takes a long time)
    spots.get_signals(volumes="all", experiment=experiment, batch_size=BATCH_SIZE, traces_type="raw")
    spots.to_json(f"spots/signals/spots_{group_tag}.json")

def get_unique_spots(spots, radius = 1): 
    """
    Returns a new spots object with the spots that are unique to each spot group.
    Spots are considered unique if they are not within a radius of another spot in any other spot group.
    """

    # collect the centroids of each spot group
    centroids = {}
    for t_tag in spots.keys():
        centroids[t_tag] = np.array(spots[t_tag]._get_centers(units='phs'))
        print(t_tag, centroids[t_tag].shape)

    # collect the indices of the nearest neighbors for each spot group
    indices_1 = {}
    for t_tag in spots.keys():
        indices_1[t_tag] = {}
        for t_tag2 in spots.keys():
            if t_tag != t_tag2:
                _, id_1, _ = nu.unique_nearest_neighbour_assignment(
                                        centroids[t_tag], centroids[t_tag2], 
                                        depth = 5, radius = radius, return_bothways = False
                                        )

                indices_1[t_tag][t_tag2] = id_1

    # find the spots that are unique to each spot group
    new_spots_2 = indices_1["1vB1"]["2vB2"] == -1
    new_spots_3 = indices_1["1vB1"]["3vB3"] == -1 
    new_spots_3 = np.logical_and(new_spots_3, indices_1["2vB2"]["3vB3"] == -1)
    new_spots_4 = indices_1["1vB1"]["4vB4"] == -1
    new_spots_4 = np.logical_and(new_spots_4, indices_1["2vB2"]["4vB4"] == -1)
    new_spots_4 = np.logical_and(new_spots_4, indices_1["3vB3"]["4vB4"] == -1)
    new_spots_5 = indices_1["1vB1"]["5vB5"] == -1
    new_spots_5 = np.logical_and(new_spots_5, indices_1["2vB2"]["5vB5"] == -1)
    new_spots_5 = np.logical_and(new_spots_5, indices_1["3vB3"]["5vB5"] == -1)
    new_spots_5 = np.logical_and(new_spots_5, indices_1["4vB4"]["5vB5"] == -1)

    # collect the unique spots in a new spots object
    all_spots_list =  copy.deepcopy(spots["1vB1"].spots)
    for spot,is_new in zip(spots["2vB2"].spots , new_spots_2):
        if is_new:
            all_spots_list.append(spot)
    for spot,is_new in zip(spots["3vB3"].spots , new_spots_3):
        if is_new:
            all_spots_list.append(spot)
    for spot,is_new in zip(spots["4vB4"].spots , new_spots_4):
        if is_new:
            all_spots_list.append(spot)
    for spot,is_new in zip(spots["5vB5"].spots , new_spots_5):
        if is_new:
            all_spots_list.append(spot)

    print(f"Number of unique spots: {len(all_spots_list)}")

    return nu.Spots(all_spots_list)

# Remove duplicate spots

First get the spots for each t-score volume and save the masks as tif files for reference

In [ ]:
spots = {}
for t_tag in T_TAGS:
    position_file = f"processed/spots/imaris/tscore_{t_tag}_Statistics/tscore_{t_tag}_Position.csv"
    diameter_file  = f"processed/spots/imaris/tscore_{t_tag}_Statistics/tscore_{t_tag}_Diameter.csv"
    spots[t_tag] = nu.Spots.from_imaris(position_file, diameter_file, resolution=RESOLUTION, units='phs')
    print(f"{t_tag}: {spots[t_tag].num_spots} spots")

    # write masks per spot group
    group = [1 for _ in range(spots[t_tag].num_spots)]
    mask = spots[t_tag].get_group_mask(group, (Z, Y, X), diameter=None, units='pix')
    tif.imwrite(f'processed/spots/masks/mask_{t_tag}.tif',
                        mask.astype(np.uint16), shape=(Z, Y, X),
                        metadata={'spacing': RESOLUTION[0], 'unit': 'um', 'axes': 'ZYX'},
                        resolution=(1 / RESOLUTION[1], 1 / RESOLUTION[2]), imagej=True)

Now remove the duplicates pairing the spots from the different t-score volumes that are closeer than a given distance (in um) 

In [ ]:
unique_spots = get_unique_spots(spots, radius = PAIRING_RADIUS)
dummy_group = [1 for i_spot in range(unique_spots.num_spots)]
unique_mask = unique_spots.get_group_mask(dummy_group, (Z, Y, X), diameter=None, units='pix')

tif.imwrite(f'processed/spots/masks/mask_merged_r{RADIUS_TAG}.tif',
                    unique_mask.astype(np.uint16), shape=(Z, Y, X),
                    metadata={'spacing': RESOLUTION[0], 'unit': 'um', 'axes': 'ZYX'},
                    resolution=(1 / RESOLUTION[1], 1 / RESOLUTION[2]), imagej=True)

# Extract the signals and save 
sorry it outputs a lot of stuff ... 

In [ ]:
tag = "merged_r3p5"
print(f'{tag}___________________________________________________________________________________________')
load_and_save_signals(unique_spots, tag)

In [ ]:
experiment.close()